## Patches Experiment with Dex-Net

In [1]:
#global imports
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import zero_one_loss, log_loss
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.qda import QDA
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/ipykernel-4.3.1-py2.7.egg/ipykernel/pylab/config.py:66: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  inline backend."""
/usr/local/lib/python2.7/dist-packages/ipykernel-4.3.1-py2.7.egg/ipykernel/pylab/config.py:71: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  'retina', 'jpeg', 'svg', 'pdf'.""")
/usr/local/lib/python2.7/dist-packages/ipykernel-4.3.1-py2.7.egg/ipykernel/pylab/config.py:85: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  use `figure_formats` instead)""")
/usr/local/lib/python2.7/dist-packages/ipykernel-4.3.1-py2.7.egg/ipykernel/pylab/config.py:95: DeprecationW

In [2]:
from util import PatchesDataLoader, PatchesSKLearner

In [3]:
#data_path = '/home/jacky/ws/patches/data/'
data_path = '/mnt/wd_ssd/projects/deep_patches/data/patches_06_11_16'

In [4]:
pdl = PatchesDataLoader(0.25, data_path, [i for i in range(1)])

In [5]:
#QDA on fc
PatchesSKLearner.train(QDA, pdl.tr, pdl.labels['fc']['tr'], pdl.t, pdl.labels['fc']['t'], "QDA")

/home/jmahler/.local/lib/python2.7/site-packages/sklearn/qda.py:125: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QDA train accuracy 0.825375170532, test accuracy 0.581570996979


QDA(priors=None, reg_param=0.0)

In [6]:
#logistic regression on fc
PatchesSKLearner.train(linear_model.LogisticRegression, pdl.tr, pdl.labels['fc']['tr'], pdl.t, 
                       pdl.labels['fc']['t'], "Logistic Regression")

Logistic Regression train accuracy 0.722601182356, test accuracy 0.598187311178


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

In [7]:
PatchesSKLearner.train(RandomForestClassifier, pdl.tr, pdl.labels['fc']['tr'], pdl.t, pdl.labels['fc']['t'], "Random Forest Clf")

Random Forest Clf train accuracy 0.991359708959, test accuracy 0.811178247734


RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0)

In [8]:
#analytical fc prediction using approx patch normals
X_w1_normals = pdl.get_partial_train_data(('w1_normal',))
X_w2_normals = pdl.get_partial_train_data(('w2_normal',))

In [9]:
def predict_single_approx_normals(w1, w2, mu = 0.5):
    alpha = np.arctan(mu)
    
    z1 = w1[2]
    z2 = w2[2]
    
    return np.arccos(-z1) < alpha and np.arccos(-z2) < alpha

def predict_approx_normals(W1, W2):
    n = W1.shape[0]
    results = []
    for i in range(n):
        results.append(predict_single_approx_normals(W1[i], W2[i]))
    return np.array(results)

In [10]:
approx_normal_tr_preds = predict_approx_normals(X_w1_normals['tr'], X_w2_normals['tr'])
approx_normal_t_preds = predict_approx_normals(X_w1_normals['t'], X_w2_normals['t'])

In [13]:
print approx_normal_t_preds[:10]

[False False False False False False False False False False]


In [11]:
print "train loss", zero_one_loss(approx_normal_tr_preds, pdl.labels['fc']['tr'], normalize=True)
print "test loss", zero_one_loss(approx_normal_t_preds, pdl.labels['fc']['t'], normalize=True)

train loss 0.387903592542
test loss 0.359516616314


In [10]:
#reg on pfc
pfc_reg = PatchesSKLearner.train(RandomForestRegressor, pdl.tr, pdl.labels['pfc_10']['tr'], pdl.t, pdl.labels['pfc_10']['t'],
      'Random Forest Regressor PFC', print_mse = True)

Random Forest Regressor PFC train accuracy 0.96422615806, test accuracy 0.797982278609
Random Forest Regressor PFC train mse 0.00363922183066, test mse 0.0205788539116


In [ ]:
#cross entropy loss for pfc
tr_pfc_preds = pfc_reg.predict(pdl.tr)
t_pfc_preds = pfc_reg.predict(pdl.t)
tr_pfc_preds_proba = np.c_[1 - tr_pfc_preds, tr_pfc_preds]
t_pfc_preds_proba = np.c_[1 - t_pfc_preds, t_pfc_preds]

In [17]:
ce_tr_pfc = log_loss(np.round(pdl.labels['pfc_10']['tr']), tr_pfc_preds_proba)
ce_t_pfc = log_loss(np.round(pdl.labels['pfc_10']['t']), t_pfc_preds_proba)
print "Random Forest Rergressor PFC train ce {0}, test ce {0}".format(ce_tr_pfc, ce_t_pfc)

Random Forest Rergressor PFC train ce 0.20100992797, test ce 0.20100992797


In [14]:
#reg on ferrari canny
ferrari_reg = PatchesSKLearner.train(RandomForestRegressor, pdl.tr, pdl.labels['ferrari']['tr'], pdl.t, pdl.labels['ferrari']['t'],
      'Random Forest Regressor Ferrari Canny', print_mse= True)

Random Forest Regressor Ferrari Canny train accuracy 0.885257381622, test accuracy 0.322818561687
Random Forest Regressor Ferrari Canny train mse 1.06544717913e-08, test mse 5.34191610085e-08


In [18]:
#thresholding pfc to classify on fc
pfc_rd_tr = np.round(pdl.labels['pfc_10']['tr']).astype('int')
pfc_rd_t = np.round(pdl.labels['pfc_10']['t']).astype('int')

In [20]:
PatchesSKLearner.train(RandomForestClassifier, pdl.tr, pfc_rd_tr, pdl.t,
      pfc_rd_t, "Random Forest Clf on Rounded PFC")

Random Forest Clf on Rounded PFC train accuracy 0.995240552784, test accuracy 0.865043867194


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)